# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nhl_correlation(): 
    # YOUR CODE HERE
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    
    column = {'Population (2016 est.)[8]': 'Population'}
    cities = cities.rename(columns = column)


    pattern_one = '(^[A-Z\d]+.*)(?=\[.*\])'
    pattern_two = '(^[A-Z\d]+.*)'

    #re.findall(pattern,'Jets[note 61]')

    def remove_notes(string):
        if len(re.findall(pattern_one,string)) > 0:
            Team_name = re.findall(pattern_one,string)
            return Team_name[0]
        elif len(re.findall(pattern_two,string)) > 0:
            Team_name = re.findall(pattern_two,string)
            return Team_name[0]
        else:
            return '0'


    #remove_notes('-')
    #remove_notes('Jets[note 61]') --> 'Jets'

    cities['NFL'] = cities['NFL'].apply(remove_notes)
    cities['MLB'] = cities['MLB'].apply(remove_notes)
    cities['NBA'] = cities['NBA'].apply(remove_notes)
    cities['NHL'] = cities['NHL'].apply(remove_notes)
    
    def seperate_teams(string):
        return list(re.findall("([A-Z0-9]+[a-z]*)",string))
    
    # Keeping only year 2018 data
    nhl_df = nhl_df[nhl_df["year"] == 2018]
    
    # Dropping division wise classification

    nhl_df.drop((nhl_df[nhl_df['GP'].map(len) > 2].index), inplace = True)
    
    # Convert numeric columns to numeric! :P

    for cols in nhl_df.columns:
        if cols != "team" and cols != "League":
            nhl_df[cols] = pd.to_numeric(nhl_df[cols])

    
    # Dictionary of nhl teams


    team_area_dict = {'Tampa Bay Lightning': 'Tampa Bay Area', 'Boston Bruins': 'Boston', 'Toronto Maple Leafs': 'Toronto',
            'Florida Panthers': 'Miami–Fort Lauderdale', "Detroit Red Wings": 'Detroit', 'Montreal Canadiens': 'Montreal',
            'Ottawa Senators': 'Ottawa', 'Buffalo Sabres': 'Buffalo', 'Washington Capitals': 'Washington, D.C.', 
            'Pittsburgh Penguins': 'Pittsburgh', 'Philadelphia Flyers': 'Philadelphia', 'Columbus Blue Jackets': 'Columbus',
            'New Jersey Devils': 'New York City', 'Carolina Hurricanes': 'Raleigh', 'New York Islanders': 'New York City',
            'New York Rangers': 'New York City', 'Nashville Predators':'Nashville', 'Winnipeg Jets': 'Winnipeg',
            'Minnesota Wild': 'Minneapolis–Saint Paul', 'Colorado Avalanche': 'Denver', 'St. Louis Blues': 'St. Louis',
            'Dallas Stars': 'Dallas–Fort Worth', 'Chicago Blackhawks': 'Chicago', 'Vegas Golden Knights': 'Las Vegas',
            'Anaheim Ducks': 'Los Angeles', 'San Jose Sharks': 'San Francisco Bay Area', 'Los Angeles Kings': 'Los Angeles',
            'Calgary Flames': 'Calgary', 'Edmonton Oilers': 'Edmonton', 'Vancouver Canucks': 'Vancouver', 
             'Arizona Coyotes': 'Phoenix'}
    
    # Removing * at the end of the team names

    def clean_team_names(string):
        return re.findall(".*(?=\*$)|.*",string)[0]


    nhl_df["team"] = nhl_df['team'].apply(clean_team_names)
    
    
    # New column with Metro areas appended
    nhl_df["Metropolitan area"] = nhl_df["team"].map(team_area_dict)
    
    # W + L games
    nhl_df["Total games"] = nhl_df["W"] + nhl_df['L']
    
    # Win/Loss ratio
    
    nhl_df['Win/Loss Ratio'] = nhl_df["W"] / nhl_df["Total games"]
    
    # Table which combines all the required data in a single dataframe
    
    final_nhl_table = nhl_df[["team",'Metropolitan area','Win/Loss Ratio']]
    final_nhl_table = final_nhl_table.set_index('Metropolitan area').sort_values(by = 'Metropolitan area')
    table = pd.merge(final_nhl_table, cities[["Metropolitan area", "Population"]], on=['Metropolitan area'], how='left')
    
    # Convert required fields to numeric
    table.set_index(['Metropolitan area','team'], inplace = True)
    table["Population"] = pd.to_numeric(table["Population"])
    table["Win/Loss Ratio"] = pd.to_numeric(table["Win/Loss Ratio"])
    
    table.reset_index(inplace = True)
    
    # table which computes average of win/loss ratio
    table_avg = table.groupby('Metropolitan area').agg({'Win/Loss Ratio': np.average})
    table_avg = table_avg.rename(columns = {'Win/Loss Ratio': 'Avg W/L ratio'})
    
    table = table.merge(table_avg , on = 'Metropolitan area', how = 'left')
    table.set_index(["Metropolitan area",'team'], inplace = True)
    
    # Truely final dataframe
    
    last_table = table[['Population','Avg W/L ratio']]
    last_table.reset_index(inplace = True)
    last_table.drop('team', axis = 1, inplace = True)
    last_table = last_table.drop_duplicates()
    
        
    
    
    
    
    #raise NotImplementedError()
    
    population_by_region = list(last_table['Population']) # pass in metropolitan area population from cities
    win_loss_by_region = list(last_table['Avg W/L ratio']) # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_correlation():
    # YOUR CODE HERE
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    
    column = {'Population (2016 est.)[8]': 'Population'}
    cities = cities.rename(columns = column)


    pattern_one = '(^[A-Z\d]+.*)(?=\[.*\])'
    pattern_two = '(^[A-Z\d]+.*)'

    #re.findall(pattern,'Jets[note 61]')

    def remove_notes(string):
        if len(re.findall(pattern_one,string)) > 0:
            Team_name = re.findall(pattern_one,string)
            return Team_name[0]
        elif len(re.findall(pattern_two,string)) > 0:
            Team_name = re.findall(pattern_two,string)
            return Team_name[0]
        else:
            return '0'


    #remove_notes('-')
    #remove_notes('Jets[note 61]') --> 'Jets'

    cities['NFL'] = cities['NFL'].apply(remove_notes)
    cities['MLB'] = cities['MLB'].apply(remove_notes)
    cities['NBA'] = cities['NBA'].apply(remove_notes)
    cities['NHL'] = cities['NHL'].apply(remove_notes)

    def seperate_teams(string):
        return list(re.findall("([A-Z0-9]+[a-z]*)",string))
    
    
    # 2018 data
    
    nba_df = nba_df[nba_df["year"] == 2018]
    
    nba_df.drop('GB',axis = 1, inplace = True)
    
    # Convert numeric columns to numeric! :P

    for cols in nba_df.columns:
        if cols != "team" and cols != "League":
            nba_df[cols] = pd.to_numeric(nba_df[cols])
            
    # clean team names
    
    def clean_team_names(string):
        x = re.findall("^[A-Z a-z 0-9]*",string)
        return x[0]
    
    
    nba_df['team'] = nba_df['team'].apply(clean_team_names)
    
    # Dictionary of nba teams


    team_area_dict_nba = {'Toronto Raptors': 'Toronto', 'Boston Celtics': 'Boston', 'Philadelphia 76ers': 'Philadelphia',
                      'Cleveland Cavaliers': 'Cleveland', 'Indiana Pacers': 'Indianapolis', 'Detroit Pistons': 'Detroit',
                      'Miami Heat': 'Miami–Fort Lauderdale', 'Milwaukee Bucks': 'Milwaukee', 
                      'Washington Wizards': 'Washington, D.C.', 'Charlotte Hornets': 'Charlotte', 
                      'New York Knicks': 'New York City', 'Brooklyn Nets': 'New York City', 'Chicago Bulls': 'Chicago',
                      'Orlando Magic': 'Orlando', 'Atlanta Hawks': 'Atlanta', 'Houston Rockets': 'Houston',
                      'Golden State Warriors': 'San Francisco Bay Area', 'Portland Trail Blazers': 'Portland',
                      'Oklahoma City Thunder': 'Oklahoma City', 'Utah Jazz': 'Salt Lake City', 
                      'New Orleans Pelicans': 'New Orleans', 'San Antonio Spurs': 'San Antonio', 
                      'Minnesota Timberwolves': 'Minneapolis–Saint Paul', 'Denver Nuggets': 'Denver',
                      'Los Angeles Clippers': 'Los Angeles', 'Los Angeles Lakers': 'Los Angeles',
                      'Sacramento Kings': 'Sacramento', 'Dallas Mavericks': 'Dallas–Fort Worth',
                      'Memphis Grizzlies': 'Memphis' ,'Phoenix Suns': 'Phoenix'}
    
    # Map teams to metro areas
    
    nba_df["Metropolitan area"] = nba_df["team"].map(team_area_dict_nba)
    
    # new table with only the required data
    
    final_nba_table = nba_df[["team",'Metropolitan area','W/L%']]
    
    # merge table with pop data
    
    final_nba_table = pd.merge(final_nba_table, cities[["Metropolitan area", "Population"]], on=['Metropolitan area'], how='left')
    
    final_nba_table.set_index(['Metropolitan area','team'], inplace = True)
    
    #Convert to numeric
    
    final_nba_table["Population"] = pd.to_numeric(final_nba_table["Population"])
    final_nba_table["W/L%"] = pd.to_numeric(final_nba_table["W/L%"])
    
    final_nba_table.reset_index(inplace = True)
    
    # calculate average W/L% of metro area
    
    table_nba_avg = final_nba_table.groupby('Metropolitan area').agg({'W/L%': np.average})
    
    table_nba_avg = table_nba_avg.rename(columns = {'W/L%': 'Avg W/L%'})
    
    final_nba_table = final_nba_table.merge(table_nba_avg , on = 'Metropolitan area', how = 'left')
    
    final_nba_table.drop('W/L%', axis = 1, inplace = True)
    
    final_nba_table.set_index(['Metropolitan area','team'], inplace = True)
    
    # contruct df for corr
    
    last_table = final_nba_table[['Population','Avg W/L%']]

    last_table.reset_index(inplace = True)
    
    # drop team names and duplicates
    last_table.drop('team', axis = 1, inplace = True)
    last_table = last_table.drop_duplicates()
    
        
    
    
    # raise NotImplementedError()
    
    population_by_region = list(last_table['Population']) # pass in metropolitan area population from cities
    win_loss_by_region = list(last_table['Avg W/L%']) # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_correlation(): 
    # YOUR CODE HERE
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    
    column = {'Population (2016 est.)[8]': 'Population'}
    cities = cities.rename(columns = column)


    pattern_one = '(^[A-Z\d]+.*)(?=\[.*\])'
    pattern_two = '(^[A-Z\d]+.*)'

    #re.findall(pattern,'Jets[note 61]')

    def remove_notes(string):
        if len(re.findall(pattern_one,string)) > 0:
            Team_name = re.findall(pattern_one,string)
            return Team_name[0]
        elif len(re.findall(pattern_two,string)) > 0:
            Team_name = re.findall(pattern_two,string)
            return Team_name[0]
        else:
            return '0'


    #remove_notes('-')
    #remove_notes('Jets[note 61]') --> 'Jets'

    cities['NFL'] = cities['NFL'].apply(remove_notes)
    cities['MLB'] = cities['MLB'].apply(remove_notes)
    cities['NBA'] = cities['NBA'].apply(remove_notes)
    cities['NHL'] = cities['NHL'].apply(remove_notes)
    
    # Kepp 2018 data
    mlb_df = mlb_df[mlb_df["year"] == 2018]
    
    team_area_dict_mlb = {'Boston Red Sox': 'Boston',
                         'New York Yankees': 'New York City',
                         'Tampa Bay Rays': 'Tampa Bay Area',
                         'Toronto Blue Jays': 'Toronto',
                         'Baltimore Orioles': "Baltimore" ,
                         'Cleveland Indians': 'Cleveland',
                         'Minnesota Twins': 'Minneapolis–Saint Paul',
                         'Detroit Tigers': 'Detroit',
                         'Chicago White Sox': 'Chicago',
                         'Kansas City Royals': 'Kansas City',
                         'Houston Astros': 'Houston',
                         'Oakland Athletics': 'San Francisco Bay Area'  ,
                         'Seattle Mariners': 'Seattle',
                         'Los Angeles Angels': 'Los Angeles',
                         'Texas Rangers': 'Dallas–Fort Worth' ,
                         'Atlanta Braves': 'Atlanta',
                         'Washington Nationals': 'Washington, D.C.',
                         'Philadelphia Phillies': 'Philadelphia',
                         'New York Mets': 'New York City',
                         'Miami Marlins': 'Miami–Fort Lauderdale' ,
                         'Milwaukee Brewers': 'Milwaukee',
                         'Chicago Cubs': 'Chicago',
                         'St. Louis Cardinals': 'St. Louis',
                         'Pittsburgh Pirates': 'Pittsburgh',
                         'Cincinnati Reds': 'Cincinnati',
                         'Los Angeles Dodgers': 'Los Angeles',
                         'Colorado Rockies': 'Denver',
                         'Arizona Diamondbacks': 'Phoenix',
                         'San Francisco Giants': 'San Francisco Bay Area',
                         'San Diego Padres': 'San Diego'}
    
    
    # Map teams to areas
    
    mlb_df["Metropolitan area"] = mlb_df["team"].map(team_area_dict_mlb)
    
    final_table_mlb = mlb_df[['Metropolitan area', 'team', 'W-L%']]
    
    final_table_mlb = pd.merge(final_table_mlb, cities[["Metropolitan area", "Population"]], on=['Metropolitan area'], how='left')
    
    # Convert required columns to numeric
    final_table_mlb['Population'] = pd.to_numeric(final_table_mlb['Population'])
    final_table_mlb['W-L%'] = pd.to_numeric(final_table_mlb['W-L%'])
    
    
    table_mlb_avg = final_table_mlb.groupby('Metropolitan area').agg({'W-L%': np.average})
    
    table_mlb_avg = table_mlb_avg.rename(columns = {'W-L%': 'Avg W/L%'})
    
    final_table_mlb = final_table_mlb.merge(table_mlb_avg , on = 'Metropolitan area', how = 'left')
    
    final_table_mlb.drop('W-L%', axis = 1, inplace = True)
    
    final_table_mlb.set_index(['Metropolitan area','team'], inplace = True)
    
    last_table = final_table_mlb[['Population','Avg W/L%']]
    last_table.reset_index(inplace = True)
    
    last_table.drop('team', axis = 1, inplace = True)
    last_table = last_table.drop_duplicates()
    
      
    
    
    # raise NotImplementedError()
    
    population_by_region = list(last_table['Population']) # pass in metropolitan area population from cities
    win_loss_by_region = list(last_table['Avg W/L%']) # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_correlation(): 
    # YOUR CODE HERE
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    
    column = {'Population (2016 est.)[8]': 'Population'}
    cities = cities.rename(columns = column)


    pattern_one = '(^[A-Z\d]+.*)(?=\[.*\])'
    pattern_two = '(^[A-Z\d]+.*)'

    #re.findall(pattern,'Jets[note 61]')

    def remove_notes(string):
        if len(re.findall(pattern_one,string)) > 0:
            Team_name = re.findall(pattern_one,string)
            return Team_name[0]
        elif len(re.findall(pattern_two,string)) > 0:
            Team_name = re.findall(pattern_two,string)
            return Team_name[0]
        else:
            return '0'


    #remove_notes('-')
    #remove_notes('Jets[note 61]') --> 'Jets'

    cities['NFL'] = cities['NFL'].apply(remove_notes)
    cities['MLB'] = cities['MLB'].apply(remove_notes)
    cities['NBA'] = cities['NBA'].apply(remove_notes)
    cities['NHL'] = cities['NHL'].apply(remove_notes)
    
    
    #########################################################
    
    
    # Select 2018 data
    
    nfl_df = nfl_df[nfl_df["year"] == 2018]
    
    def clean_team_names(string):
        return re.findall(".*(?=[\*\+]$)|.*",string)[0]


    nfl_df['team'] = nfl_df['team'].apply(clean_team_names)
    
    ############################################################
    
    # Dropping division wise classification

    nfl_df = nfl_df.drop((nfl_df[nfl_df['W'].map(len) > 2].index))
    
    
    nfl_df = nfl_df[['team', 'W', 'L', 'W-L%']]
    
    nfl_df['W'] = pd.to_numeric(nfl_df['W'])
    nfl_df['L'] = pd.to_numeric(nfl_df['L'])
    nfl_df['W-L%'] = pd.to_numeric(nfl_df['W-L%'])
    
    
    ##############################################################
    
    team_area_dict_nfl = {'New England Patriots': 'Boston',
                         'Miami Dolphins': 'Miami–Fort Lauderdale',
                         'Buffalo Bills': 'Buffalo',
                         'New York Jets': 'New York City',
                         'Baltimore Ravens': 'Baltimore',
                         'Pittsburgh Steelers': 'Pittsburgh',
                         'Cleveland Browns': 'Cleveland',
                         'Cincinnati Bengals': 'Cincinnati',
                         'Houston Texans': 'Houston',
                         'Indianapolis Colts': 'Indianapolis',
                         'Tennessee Titans': 'Nashville',
                         'Jacksonville Jaguars':'Jacksonville',
                         'Kansas City Chiefs': 'Kansas City',
                         'Los Angeles Chargers': 'Los Angeles',
                         'Denver Broncos': 'Denver',
                         'Oakland Raiders': 'San Francisco Bay Area' ,
                         'Dallas Cowboys': 'Dallas–Fort Worth' ,
                         'Philadelphia Eagles': 'Philadelphia',
                         'Washington Redskins': 'Washington, D.C.',
                         'New York Giants': 'New York City',
                         'Chicago Bears': 'Chicago',
                         'Minnesota Vikings': 'Minneapolis–Saint Paul',
                         'Green Bay Packers': 'Green Bay',
                         'Detroit Lions': 'Detroit',
                         'New Orleans Saints': 'New Orleans',
                         'Carolina Panthers': 'Charlotte' ,
                         'Atlanta Falcons': 'Atlanta',
                         'Tampa Bay Buccaneers': 'Tampa Bay Area',
                         'Los Angeles Rams': 'Los Angeles',
                         'Seattle Seahawks': 'Seattle',
                         'San Francisco 49ers': 'San Francisco Bay Area' ,
                         'Arizona Cardinals': 'Phoenix'}
    
    ####################################################################
    
    nfl_df["Metropolitan area"] = nfl_df["team"].map(team_area_dict_nfl)
    final_table_nfl = nfl_df[['Metropolitan area', 'team', 'W-L%']]
    final_table_nfl = pd.merge(final_table_nfl, cities[["Metropolitan area", "Population"]], on=['Metropolitan area'], how='left')
    
    table_nfl_avg = final_table_nfl.groupby('Metropolitan area').agg({'W-L%': np.average})
    table_nfl_avg = table_nfl_avg.rename(columns = {'W-L%': 'Avg W/L%'})
    final_table_nfl = final_table_nfl.merge(table_nfl_avg , on = 'Metropolitan area', how = 'left')
    
    final_table_nfl.drop('W-L%', axis = 1, inplace = True)
    final_table_nfl.set_index(['Metropolitan area','team'], inplace = True)
    
    
    #####################################################################
    
    last_table = final_table_nfl[['Population','Avg W/L%']]

    last_table.reset_index(inplace = True)
    last_table.drop('team', axis = 1, inplace = True)
    last_table = last_table.drop_duplicates()
    
    last_table["Population"] = pd.to_numeric(last_table["Population"])
    
    
    
    
    
    # raise NotImplementedError()
    
    population_by_region = list(last_table['Population']) # pass in metropolitan area population from cities
    win_loss_by_region = list(last_table['Avg W/L%']) # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [5]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values